# MODELLING - REGRESSION

Goal: choose the algorithm that performs better and its hyperparameters.


**IMPORTANT:** This stage is designed for a high-level (maximum-scope) modeling view, in which we compare many algorithms and also many parameters.
If we experience memory or performance issues, we should reduce the problem by:
- Sampling the data
- Applying undersampling for balancing
- Reducing the number of algorithms to test
- Reducing the number of parameters to test
- Using random search and specifying an appropriate n_iter

## IMPORT LIBRARIES

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Autocomplete
%config IPCompleter.greedy=True

# Disable scientific notation
pd.options.display.float_format = '{:.2f}'.format

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

## IMPORT DATA

In [2]:
PROJECT_PATH = '/Users/rober/cmapss-rul-prediction/'

In [3]:
name_X = 'X_preselected.pickle'
name_y = 'y_preselected.pickle'

In [4]:
X = pd.read_pickle(PROJECT_PATH + '/02_Data/03_Working/' + name_X)
y = pd.read_pickle(PROJECT_PATH + '/02_Data/03_Working/' + name_y)

## MODELLING

### Set aside the validation dataset

We'll stick to the common approach of **50/30/20 (train/test/validation)**.

This approach prioritizes reliable evaluation and minimizes overfitting, while still leaving enough data for effective model learning given the dataset’s size (~14K rows).

In [8]:
train_X,val_X,train_y,val_y = train_test_split(X,y,test_size=0.3)

### Create the pipeline and the dictionary of algorithms, parameters, and values to test.

Modify to keep only the algorithms you want to test.

Adjust the parameters accordingly.

In [10]:
pipe = Pipeline([('algorithm', RandomForestRegressor())])

grid = [
    {'algorithm': [LinearRegression()],
     'algorithm__n_jobs': [-1]},
    
    {'algorithm': [RandomForestRegressor()],
     'algorithm__n_jobs': [-1],
     'algorithm__max_depth': [5, 10, 15],
     'algorithm__n_estimators': [50, 100, 200]},
    
    {'algorithm': [XGBRegressor()],
     'algorithm__n_jobs': [-1],
     'algorithm__learning_rate': [0.01, 0.025, 0.05, 0.1],
     'algorithm__max_depth': [5, 10, 20],
     'algorithm__reg_alpha': [0, 0.1, 0.5, 1],
     'algorithm__reg_lambda': [0.01, 0.1, 1],
     'algorithm__n_estimators': [100, 500, 1000]},
    
    {'algorithm': [HistGradientBoostingRegressor()],
     'algorithm__learning_rate': [0.01, 0.025, 0.05, 0.1],
     'algorithm__max_iter': [50, 100, 200],
     'algorithm__max_depth': [5, 10, 20],
     'algorithm__min_samples_leaf': [500],
     'algorithm__scoring': ['neg_mean_absolute_percentage_error'],
     'algorithm__l2_regularization': [0, 0.25, 0.5, 0.75, 1]}
]

### Optimize the hyperparameters

We'll use **RandomizedSearchCV** instead of GridSearch because:
- It’s faster — it tests only a random subset of all combinations instead of every single one.
- It still gives excellent results if you set a reasonable n_iter (e.g., 20–50).
- It’s ideal when you have many parameters or large grids — like in your setup with several algorithms and many tuning values.

####  Grid search

In [33]:
# grid_search = GridSearchCV(estimator= pipe, 
#                            param_grid = grid, 
#                            cv = 3, 
#                            scoring = 'neg_mean_absolute_percentage_error',
#                            verbose = 0,
#                            n_jobs = -1)

# modelo = grid_search.fit(train_X,train_y)

# pd.DataFrame(grid_search.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algoritmo,param_algoritmo__n_jobs,param_algoritmo__max_depth,param_algoritmo__n_estimators,param_algoritmo__learning_rate,param_algoritmo__reg_alpha,...,param_algoritmo__max_iter,param_algoritmo__min_samples_leaf,param_algoritmo__scoring,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.02,0.00,0.00,0.00,LinearRegression(n_jobs=-1),-1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"{'algoritmo': LinearRegression(n_jobs=-1), 'al...",-0.24,-0.14,-0.24,-0.21,0.05,1
352,3.97,0.16,0.01,0.00,"XGBRegressor(base_score=None, booster=None, co...",-1,5,500,0.10,0.50,...,NaN,NaN,NaN,"{'algoritmo': XGBRegressor(base_score=None, bo...",-0.52,-0.70,-1.70,-0.97,0.52,2
364,4.50,0.18,0.01,0.00,"XGBRegressor(base_score=None, booster=None, co...",-1,5,1000,0.10,0.50,...,NaN,NaN,NaN,"{'algoritmo': XGBRegressor(base_score=None, bo...",-0.52,-0.70,-1.70,-0.97,0.52,2
360,7.80,0.13,0.01,0.00,"XGBRegressor(base_score=None, booster=None, co...",-1,5,1000,0.10,0,...,NaN,NaN,NaN,"{'algoritmo': XGBRegressor(base_score=None, bo...",-0.57,-0.74,-1.61,-0.97,0.46,4
348,4.18,0.13,0.01,0.00,"XGBRegressor(base_score=None, booster=None, co...",-1,5,500,0.10,0,...,NaN,NaN,NaN,"{'algoritmo': XGBRegressor(base_score=None, bo...",-0.57,-0.74,-1.61,-0.97,0.46,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,4.57,0.11,0.01,0.00,"XGBRegressor(base_score=None, booster=None, co...",-1,20,100,0.05,0.10,...,NaN,NaN,NaN,"{'algoritmo': XGBRegressor(base_score=None, bo...",-3.72,-1.70,-2.71,-2.71,0.82,618
313,8.24,0.12,0.01,0.00,"XGBRegressor(base_score=None, booster=None, co...",-1,20,500,0.05,0.10,...,NaN,NaN,NaN,"{'algoritmo': XGBRegressor(base_score=None, bo...",-3.73,-1.71,-2.72,-2.72,0.82,619
325,11.82,0.59,0.01,0.00,"XGBRegressor(base_score=None, booster=None, co...",-1,20,1000,0.05,0.10,...,NaN,NaN,NaN,"{'algoritmo': XGBRegressor(base_score=None, bo...",-3.73,-1.71,-2.72,-2.72,0.82,619
217,17.26,0.41,0.01,0.00,"XGBRegressor(base_score=None, booster=None, co...",-1,20,1000,0.03,0.10,...,NaN,NaN,NaN,"{'algoritmo': XGBRegressor(base_score=None, bo...",-3.85,-1.70,-2.72,-2.76,0.88,621


####  Random search

In [20]:
random_search = RandomizedSearchCV(estimator = pipe,
                                   param_distributions = grid, 
                                   n_iter = 25, 
                                   cv = 3, 
                                   scoring = 'neg_root_mean_squared_error', 
                                   verbose = 0,
                                   n_jobs = -1)

model = random_search.fit(train_X,train_y)

pd.DataFrame(random_search.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm__scoring,param_algorithm__min_samples_leaf,param_algorithm__max_iter,param_algorithm__max_depth,param_algorithm__learning_rate,param_algorithm__l2_regularization,...,param_algorithm__reg_alpha,param_algorithm__n_jobs,param_algorithm__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
13,0.50,0.12,0.05,0.00,neg_mean_absolute_percentage_error,500,200,10,0.03,0.50,...,NaN,NaN,NaN,{'algorithm__scoring': 'neg_mean_absolute_perc...,-39.81,-39.35,-39.51,-39.56,0.19,1
20,0.37,0.01,0.05,0.00,neg_mean_absolute_percentage_error,500,200,10,0.10,0.75,...,NaN,NaN,NaN,{'algorithm__scoring': 'neg_mean_absolute_perc...,-40.15,-39.46,-39.67,-39.76,0.29,2
7,1.39,0.03,0.05,0.00,NaN,NaN,NaN,5,0.01,NaN,...,0,-1,500,"{'algorithm__reg_lambda': 0.1, 'algorithm__reg...",-40.18,-39.61,-39.77,-39.86,0.24,3
8,1.37,0.10,0.06,0.01,NaN,NaN,NaN,5,0.01,NaN,...,0,-1,500,"{'algorithm__reg_lambda': 0.01, 'algorithm__re...",-40.18,-39.60,-39.80,-39.86,0.24,4
4,2.44,0.27,0.20,0.09,NaN,NaN,NaN,5,0.01,NaN,...,0.10,-1,1000,"{'algorithm__reg_lambda': 0.1, 'algorithm__reg...",-40.59,-39.97,-40.09,-40.22,0.27,5
16,0.36,0.00,0.04,0.00,neg_mean_absolute_percentage_error,500,200,20,0.01,1,...,NaN,NaN,NaN,{'algorithm__scoring': 'neg_mean_absolute_perc...,-40.68,-40.20,-40.76,-40.55,0.25,6
12,15.45,3.15,0.23,0.06,NaN,NaN,NaN,10,0.01,NaN,...,0.10,-1,500,"{'algorithm__reg_lambda': 1, 'algorithm__reg_a...",-42.00,-40.89,-41.66,-41.52,0.46,7
19,13.35,0.54,0.22,0.03,NaN,NaN,NaN,10,0.01,NaN,...,0.10,-1,500,"{'algorithm__reg_lambda': 0.01, 'algorithm__re...",-42.61,-41.31,-41.59,-41.83,0.56,8
14,2.06,0.04,0.03,0.01,NaN,NaN,NaN,10,0.10,NaN,...,0,-1,100,"{'algorithm__reg_lambda': 1, 'algorithm__reg_a...",-42.69,-41.40,-41.98,-42.03,0.53,9
22,20.41,0.50,0.38,0.02,NaN,NaN,NaN,10,0.01,NaN,...,1,-1,1000,"{'algorithm__reg_lambda': 0.1, 'algorithm__reg...",-42.97,-41.56,-41.74,-42.09,0.63,10


## EVALUATE

### Predict over validation

In [13]:
pred = model.best_estimator_.predict(val_X)

### Evaluate over validation

In [18]:
mae = mean_absolute_error(val_y, pred)
rmse = np.sqrt(mean_squared_error(val_y, pred))
r2 = r2_score(val_y, pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")

MAE: 30.81
RMSE: 43.98
R²: 0.620


Target (RUL) range: 0-250

| **Metric** | **Ideal Range** | **Your Value** | **Interpretation Level** | **Quick Explanation (anchored to 0–250)**                                                                                                                                                                              |
| ---------- | --------------- | -------------- | ------------------------ | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **MAE**    | < 10% of range  | **30.8**       | 🟢 Good       | **31 cycles**, ~**12%** of the 0–250 range ⇒ solid accuracy                                                                                                      |
| **RMSE**   | < 15% of range  | **43.9**       | 🟢 Good baseline         | **44 cycles**, ~**17.6%** of the 0–250 range. RMSE penalizes big mistakes more than MAE; here it’s ~1.4× MAE, which is normal (if RMSE were 2× MAE or higher, it’d mean huge outliers) ⇒ some bigger errors exist but aren’t dominant                                   |
| **R²**     | > 0.6 is solid  | **0.62**       | 🟢 Moderate–good fit     | **62%** of variance explained (0 = just predicting the mean; 1 = perfect) **0.62** means the model captures a **meaningful chunk** of the pattern but still misses ~38% of variability ⇒ room for improvement |

#### Ad-hoc: 📏 Typical Regression Metric Ranges (generic reference)

| **Metric** | **Excellent**          | **Good**  | **Acceptable** | **Poor** |
| ---------- | ---------------------- | --------- | -------------- | -------- |
| **MAE**    | < 10 % of target range | 10–15 %   | 15–25 %        | > 25 %   |
| **RMSE**   | < 15 % of target range | 15–20 %   | 20–30 %        | > 30 %   |
| **R²**     | > 0.80                 | 0.60–0.80 | 0.40–0.60      | < 0.40   |



**⚙️ Why R² ≈ 0.6 and MAE ≈ 12 % are “good” in RUL prediction**

- **The data are noisy** — In CMAPSS and other RUL datasets, sensors capture complex, real-world engine behavior — lots of randomness, environmental variation, and wear patterns that aren’t fully predictable. Even the best models can’t explain all that variance.
- **The process is nonlinear and multi-factor** — RUL depends on many interacting variables (temperature, pressure, vibration, etc.). That makes perfect prediction (R² → 1) practically impossible — 0.6 – 0.8 is already strong.
- **Targets often saturate at zero** — Since RUL stops at 0 (failure), you get skewed distributions and hard-to-fit tails. That limits how “tight” your MAE or RMSE can realistically be.
- **Baseline comparison** — If you just predicted the mean RUL for everything, R² would be 0. So a model explaining ~60 % of the variance (R² = 0.6) is capturing most of the useful signal.

For noisy, nonlinear, real-world problems like RUL prediction, a model that explains ~60 % of variance and stays within ~10–15 % average error of the true range is considered very good — it means the model learned real, actionable patterns instead of random noise.

### Best model

In [22]:
model.best_estimator_

Pipeline(steps=[('algorithm',
                 HistGradientBoostingRegressor(l2_regularization=0.5,
                                               learning_rate=0.025,
                                               max_depth=10, max_iter=200,
                                               min_samples_leaf=500,
                                               scoring='neg_mean_absolute_percentage_error'))])

HistGradientBoostingRegressor(l2_regularization=0.5,
                                               learning_rate=0.025,
                                               max_depth=10, max_iter=200,
                                               min_samples_leaf=500,
                                               scoring='neg_mean_absolute_percentage_error')